<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/ex09/rodrigo_cabrera_castaldoni/Aula_9_Exercicio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
nome = "Rodrigo Cabrera Castaldoni"
print(f'Meu nome é {nome}')

Meu nome é Rodrigo Cabrera Castaldoni


#  Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 8, mas iremos agora treinar uma rede neural com **duas camadas** de auto-atenção **causais** para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Iremos também trabalhar com sequencias de tamanho variável.

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)


O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [ ]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Importação dos pacotes

In [ ]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook


In [ ]:
# Check which GPU we are using
!nvidia-smi

Sun Jun  5 20:38:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
if torch.cuda.is_available():
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [ ]:
from typing import List


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, max_seq_length: int):
        # Escreva aqui seu código.
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length 
        self.start_sequence_id = self.tokenizer.cls_token_id
        self.dataset = self._create_dataset()

    def _tokenize(self, text: str, tokenizer):
        # Recomenda-se usar o tokenizer.batch_encode_plus pois é mais rápido.
        return tokenizer.batch_encode_plus(text, return_tensors=None, add_special_tokens=False).input_ids

    def _text2samples(self, text):
      tokens = tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids

      num_examples, resto = divmod(len(tokens), self.max_seq_length-1)
      num_examples += 1 if resto!= 0 else 0
      total_elem = num_examples*(self.max_seq_length -1)

      padded_text = nn.functional.pad(torch.tensor(tokens), (0, total_elem - len(tokens)))
      padded_text = padded_text.reshape(num_examples,-1)

      init_sequence = torch.ones((num_examples, 1))*torch.tensor(self.start_sequence_id)
      return torch.cat((init_sequence, padded_text), dim=1).type(torch.LongTensor)

    def _create_dataset(self):
        return torch.cat([self._text2samples(text) for text in tqdm_notebook(self.texts)])

    def __len__(self):
        # Escreva aqui seu código.
        return len(self.dataset)

    def __getitem__(self, idx):
        # Escreva aqui seu código.
        """
        Eu acho que não estou otimizando memória aqui ...
        """
        target = nn.functional.pad(self.dataset[idx, 1:], (0, 1))
        input = self.dataset[idx, :]

        return input, target

In [ ]:
# def split_sequence(sequence_tokens, start_sequence_id, max_seq_length):

#   tokens = start_sequence_id + sequence_tokens
#   if len(tokens) < max_seq_length:
#     return [np.pad(tokens, (0, max_seq_length - len(tokens)))]
#   else:
#     first_part = np.asarray(tokens[:max_seq_length])
#     last_part = tokens[max_seq_length:]
#     return [first_part] + split_sequence(last_part, start_sequence_id, max_seq_length)


# sample_text = 'Eu gosto de correr muito'
# tokens = tokenizer(sample_text, return_tensors=None, add_special_tokens=False).input_ids
# start_sequence_id = tokenizer("[CLS]", return_tensors=None, add_special_tokens=False).input_ids

# print(tokens)
# split_sequence(tokens, start_sequence_id, 4)

## Testando se a implementação do MyDataset está correta

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, max_seq_length=9)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 2
print('Passou no assert de tamanho do dataset.')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[  101,  3396, 10303,   125, 13239,     0,     0,     0,     0],
     [  101,  1660,  5971,   785,   125,  1847, 13779, 15616,     0]])

correct_first_batch_target = torch.LongTensor(
    [[ 3396, 10303,   125, 13239,     0,     0,     0,     0,     0],
     [ 1660,  5971,   785,   125,  1847, 13779, 15616,     0,     0]])

assert torch.equal(first_batch_input, correct_first_batch_input)
assert torch.equal(first_batch_target, correct_first_batch_target)

print('Passou no assert de dataset.')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/2 [00:00<?, ?it/s]

Passou no assert de tamanho do dataset.
Passou no assert de dataset.


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [ ]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt

File ‘sample-1gb.txt’ already there; not retrieving.



In [ ]:
# Load datasets
max_seq_length = 12

train_examples = 50000
valid_examples = 100
test_examples = 100

texts = open('sample-1gb.txt').readlines()

print(f'Read {len(texts)} lines.')

max_lines = train_examples + valid_examples + test_examples
print(f'Truncating to {max_lines} lines.')
texts = texts[:max_lines]  

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)

Read 250000 lines.
Truncating to 50200 lines.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/50000 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 5065773
valid examples: 11409
test examples: 11438


In [ ]:
class CausalAttentionHead(torch.nn.Module):

  def __init__(self, mask, pad_token_id, embedding_dim, head_dim):
    super(CausalAttentionHead, self).__init__()
    self.mask = mask
    self.pad_token_id = pad_token_id
    self.head_dim = torch.tensor([head_dim], device=device) # somente um escalar para realizar a divisão.
    self.WQ = nn.Linear(embedding_dim, head_dim, bias=False)
    self.WK = nn.Linear(embedding_dim, head_dim, bias=False)
    self.WV = nn.Linear(embedding_dim, head_dim, bias=False)

  def forward(self, inputs):

    keys = self.WK(inputs) # size: (batch_size, num_tokens, head_dim)
    querys = self.WQ(inputs) # size: (batch_size, num_tokens, head_dim)
    values = self.WV(inputs) # size: (batch_size, num_tokens, head_dim)

    # aplica multiplicação matricial em batch 
    scores = torch.bmm(querys, keys.transpose(dim0=1, dim1=2)) / torch.sqrt(self.head_dim) # size: (batch_size, num_tokens, num_tokens)
    masked_scores = scores.masked_fill(self.mask == self.pad_token_id, -float("inf")) # size: (batch_size, num_tokens, num_tokens); truque para dar atenção aos tokens corretos
    weights = torch.nn.functional.softmax(masked_scores, dim=-1) # size: (batch_size, num_tokens, num_tokens); transforma os scores em probas ao longo do último eixo
    attention_scores = torch.bmm(weights, values) # size: (batch_size, num_tokens, head_dim)
    
    return attention_scores

class TransformerLayer(torch.nn.Module):

  def __init__(self, mask, dim: int, pad_token_id: int, num_attention_heads: int):
    super(TransformerLayer, self).__init__()

    # Encontra a dimensão correta de cada cabeça 
    head_dim = dim // num_attention_heads

    # Constrói mascara do AttentionHeadLayer

    # Encontra a dimensão correta de cada cabeça 

    self.attentions_heads = nn.ModuleList([CausalAttentionHead(mask, pad_token_id, dim, head_dim) for _ in range(num_attention_heads)])
    self.WO = nn.Linear(dim, dim)
    self.layer_norm1 = nn.LayerNorm(dim)
    self.layer_norm2 = nn.LayerNorm(dim)

    """
    A rule of thumb from the literature is for the hidden size of the first layer to be four times the size of the embeddings.

    Essa aqui é uma position-wise feed-forward layer.
    """

    self.feed_forward = nn.Sequential(
        torch.nn.Linear(dim, 2*dim),
        torch.nn.ReLU(),
        torch.nn.Linear(2*dim, dim),
        nn.Dropout(0.3)
    )

  def forward(self, inputs): 

    # No caso em que o numero de cabecas seja multiplo de embedding_dim
    concat_heads = torch.cat([h(inputs) for h in self.attentions_heads], dim=-1) # size: (batch_size, num_tokens, embedding_dim) 

    out = self.layer_norm1(self.WO(concat_heads) + inputs)
    
    return self.layer_norm2(self.feed_forward(out) + inputs)

In [ ]:
class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size: int, max_seq_length: int, dim: int, n_layers: int, pad_token_id: int, num_attention_heads):
      """
      Implements the Self-attention, decoder-only."

      Args:
          vocab_size (int): Size of the input vocabulary.
          max_seq_length (int): Size of the sequence to consider as context for prediction.
          dim (int): Dimension of the embedding layer for each word in the context.
          n_layers (int): number of self-attention layers.
          pad_token_id (int): id of the pad token that will be ignored in the attention.
      """
      # Escreva seu código aqui.
      super(LanguageModel, self).__init__()
      self.pad_token_id = pad_token_id
      self.token_embeddings = nn.Embedding(vocab_size, dim, padding_idx=pad_token_id)
      self.position_embeddings = nn.Embedding(max_seq_length, dim)     
      self.position_ids = torch.arange(max_seq_length, dtype=torch.long, device=device).unsqueeze(0)

      mask = torch.tril(torch.ones(max_seq_length, max_seq_length)).unsqueeze(0).to(device)

      self.transformer_layer = nn.ModuleList([TransformerLayer(mask, dim, pad_token_id, num_attention_heads) for _ in range(n_layers)])

      self.classifier = nn.Sequential(
        torch.nn.Linear(dim, 2*dim),
        torch.nn.ReLU(),
        torch.nn.Linear(2*dim, 4*dim),
        torch.nn.ReLU(),
        torch.nn.Linear(4*dim, vocab_size),
      )


    def forward(self, inputs):
      """
      Args:
          inputs is a LongTensor of shape (batch_size, max_seq_length)
          
      Returns:
          logits of shape (batch_size, max_seq_length, vocab_size)
      """
      # Escreva seu código aqui.
      token_embeddings = self.token_embeddings(inputs) 
      position_embeddings = self.position_embeddings(self.position_ids)
      x = position_embeddings + token_embeddings
      for layer in self.transformer_layer:
        x = layer(x)
      
      return self.classifier(x)


In [ ]:
model.pad_token_id

0

## Teste o modelo com um exemplo

In [ ]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=64,
    n_layers=1,
    pad_token_id=tokenizer.pad_token_id,
    num_attention_heads=32
).to(device)

sample_input, _ = next(iter(DataLoader(training_dataset,batch_size=2)))
sample_input = sample_input.to(device)
sample_output = model(sample_input)
print(f'sample_input.shape: {sample_input.shape}')
print(f'sample_output.shape: {sample_output.shape}')

sample_input.shape: torch.Size([2, 12])
sample_output.shape: torch.Size([2, 12, 29794])


In [ ]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 9639266


## Assert da Perplexidade


In [ ]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target, ignore_token_id: int):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, seq_length, vocab_size)
        target: a LongTensor of shape (batch_size, seq_length)

    Returns:
        A float corresponding to the perplexity
    """
    logits = logits.reshape(-1, logits.shape[-1])
    target = target.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target, reduction='mean', ignore_index=ignore_token_id)
    return torch.exp(loss)


n_examples = 1000

train_input_ids, train_target_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
train_input_ids = train_input_ids.to(device)
train_target_ids = train_target_ids.to(device)

logits = model(train_input_ids)

my_perplexity = perplexity(logits=logits, target=train_target_ids, ignore_token_id=tokenizer.pad_token_id)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

my perplexity:              30106
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [ ]:
max_examples = 150_000_000
eval_every_steps = 1000
lr = 3e-4


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=256,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
    num_attention_heads=32    
).to(device)

train_loader = DataLoader(training_dataset, batch_size=64, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=64)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for train_input_ids, train_target_ids in train_loader:
        loss = train_step(train_input_ids.to(device), train_target_ids.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(val_input_ids.to(device), val_target_ids.to(device))
                    for val_input_ids, val_target_ids in validation_loader]))

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(train_input_ids)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 30262.83, valid ppl: 29022.21
1000 steps; 64000 examples so far; train ppl: 1318.33, valid ppl: 921.44
2000 steps; 128000 examples so far; train ppl: 762.20, valid ppl: 674.43
3000 steps; 192000 examples so far; train ppl: 591.41, valid ppl: 534.90
4000 steps; 256000 examples so far; train ppl: 491.28, valid ppl: 458.66
5000 steps; 320000 examples so far; train ppl: 435.20, valid ppl: 414.26
6000 steps; 384000 examples so far; train ppl: 395.66, valid ppl: 381.50
7000 steps; 448000 examples so far; train ppl: 371.79, valid ppl: 359.97
8000 steps; 512000 examples so far; train ppl: 351.45, valid ppl: 343.97
9000 steps; 576000 examples so far; train ppl: 338.41, valid ppl: 328.73
10000 steps; 640000 examples so far; train ppl: 326.79, valid ppl: 317.31
11000 steps; 704000 examples so far; train ppl: 317.30, valid ppl: 309.30
12000 steps; 768000 examples so far; train ppl: 309.38, valid ppl: 302.05
13000 steps; 832000 examples so far; train ppl: 301.

KeyboardInterrupt: ignored

## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(test_input_ids.to(device), test_target_ids.to(device))
        for test_input_ids, test_target_ids in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 178.48945769210718


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [ ]:
def tokenize(text: str, tokenizer):
    # Recomenda-se usar o tokenizer.batch_encode_plus pois é mais rápido.
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids

In [ ]:
prompt = 'Eu gosto de comer pizza pois me faz bem comer aos'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de comer pizza pois me faz bem comer aos que
Eu gosto de comer pizza pois me faz bem comer aos que a
Eu gosto de comer pizza pois me faz bem comer aos que a minha
Eu gosto de comer pizza pois me faz bem comer aos que a minhas
Eu gosto de comer pizza pois me faz bem comer aos que a minhas,
Eu gosto de comer pizza pois me faz bem comer aos que a minhas, a
Eu gosto de comer pizza pois me faz bem comer aos que a minhas, a vida
Eu gosto de comer pizza pois me faz bem comer aos que a minhas, a vidamel
Eu gosto de comer pizza pois me faz bem comer aos que a minhas, a vidameli
Eu gosto de comer pizza pois me faz bem comer aos que a minhas, a vidameli,
Eu gosto de comer pizza pois me faz bem comer aos que a minhas, a vidameli, o
Eu gosto de comer pizza pois me faz bem comer aos que a minhas, a vidameli, o que
Eu gosto de comer pizza pois me faz bem comer aos que a minhas, a vidameli, o que a
Eu gosto de comer pizza pois me faz bem comer aos que a minhas, a vidameli, o que ai
Eu gosto d

## Bonus 1
Quem conseguir a menor perplexidade no dataset de testes ganha 0.5 ponto na média final.

## Bonus 2
Qual é a complexidade (em notação O-grande) da função de geração de texto acima?

Quem responder corretamente a pergunta acima e deixar a função com menor complexidade ganha 0.5 ponto na média final.